In [40]:
# Dependencies and Setup
import os
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [41]:
#read the data sets from Resources file
csv_path = "Resources/Zillow Observed Rent Index, smoothed, seasonally adjusted.csv"
rent_df = pd.read_csv(csv_path)


In [42]:
csv_path = "Resources/Zillow House Value Index, smoothed, seaonally adjusted.csv"
house_df = pd.read_csv(csv_path)

In [43]:
## rent data cleaning ##

In [44]:
#check data
rent_df.head()

,RegionID,RegionName,SizeRank,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,...,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01
0,102001,United States,0,1373.0,1380.0,1388.0,1395.0,1402.0,1409,1417,...,1744,1745,1745,1746,1746,1747,1747,1747,1747.0,1748.0
1,394913,"New York, NY",1,2379.0,2391.0,2403.0,2415.0,2426.0,2438,2450,...,2756,2734,2713,2691,2669,2646,2624,2600,2576.0,2552.0
2,753899,"Los Angeles-Long Beach-Anaheim, CA",2,1831.0,1848.0,1865.0,1882.0,1899.0,1916,1932,...,2566,2565,2564,2563,2561,2559,2558,2556,2554.0,2553.0
3,394463,"Chicago, IL",3,1511.0,1517.0,1522.0,1528.0,1533.0,1539,1544,...,1766,1762,1758,1754,1750,1746,1742,1737,1732.0,1728.0
4,394514,"Dallas-Fort Worth, TX",4,1199.0,1207.0,1214.0,1221.0,1229.0,1236,1243,...,1566,1568,1571,1573,1575,1577,1580,1582,1585.0,1587.0


In [45]:
#depivot the table from the time series
rent_df = pd.melt(rent_df, id_vars=['RegionID', 'RegionName','SizeRank'], var_name='Date', value_name='Price')

In [46]:
rent_df

,RegionID,RegionName,SizeRank,Date,Price
0,102001,United States,0,2014-01,1373.0
1,394913,"New York, NY",1,2014-01,2379.0
2,753899,"Los Angeles-Long Beach-Anaheim, CA",2,2014-01,1831.0
3,394463,"Chicago, IL",3,2014-01,1511.0
4,394514,"Dallas-Fort Worth, TX",4,2014-01,1199.0
...,...,...,...,...,...
9005,394995,"Port St. Lucie, FL",119,2021-01,1815.0
9006,394602,"Fort Collins, CO",159,2021-01,1618.0
9007,394405,"Boulder, CO",162,2021-01,2020.0
9008,394645,"Greeley, CO",177,2021-01,1681.0


In [47]:
#check for null values
rent_df.isnull().sum()

RegionID       0
RegionName     0
SizeRank       0
Date           0
Price         17
dtype: int64

In [48]:
#drop the null values
    ##included the null values might lead to inaccurate results
rent_df = rent_df.dropna()

In [49]:
rent_df.isnull().sum()

RegionID      0
RegionName    0
SizeRank      0
Date          0
Price         0
dtype: int64

In [50]:
#data types check
rent_df.dtypes

RegionID        int64
RegionName     object
SizeRank        int64
Date           object
Price         float64
dtype: object

In [51]:
#convert data types for further analysis
rent_df['Price'] = rent_df['Price'].astype(int)

In [52]:
#split Date into Year and Month columns 
rent_df[['Year','Month']] = rent_df.Date.str.split("-",expand=True)
#split RegionName into State and City columns 
rent_df[['State','City']] = rent_df.RegionName.str.split(",",expand=True)
rent_df

,RegionID,RegionName,SizeRank,Date,Price,Year,Month,State,City
0,102001,United States,0,2014-01,1373,2014,01,United States,None
1,394913,"New York, NY",1,2014-01,2379,2014,01,New York,NY
2,753899,"Los Angeles-Long Beach-Anaheim, CA",2,2014-01,1831,2014,01,Los Angeles-Long Beach-Anaheim,CA
3,394463,"Chicago, IL",3,2014-01,1511,2014,01,Chicago,IL
4,394514,"Dallas-Fort Worth, TX",4,2014-01,1199,2014,01,Dallas-Fort Worth,TX
...,...,...,...,...,...,...,...,...,...
9005,394995,"Port St. Lucie, FL",119,2021-01,1815,2021,01,Port St. Lucie,FL
9006,394602,"Fort Collins, CO",159,2021-01,1618,2021,01,Fort Collins,CO
9007,394405,"Boulder, CO",162,2021-01,2020,2021,01,Boulder,CO
9008,394645,"Greeley, CO",177,2021-01,1681,2021,01,Greeley,CO


In [53]:
#adjust the data frame columns
rent_df = rent_df[['RegionID','SizeRank','State','City','Year', 'Month','Price']]
rent_df.head()


,RegionID,SizeRank,State,City,Year,Month,Price
0,102001,0,United States,None,2014,01,1373
1,394913,1,New York,NY,2014,01,2379
2,753899,2,Los Angeles-Long Beach-Anaheim,CA,2014,01,1831
3,394463,3,Chicago,IL,2014,01,1511
4,394514,4,Dallas-Fort Worth,TX,2014,01,1199


In [54]:
#year data frames
ry14_df = rent_df.loc[rent_df["Year"] == "2014"]
ry14_df

,RegionID,SizeRank,State,City,Year,Month,Price
0,102001,0,United States,None,2014,01,1373
1,394913,1,New York,NY,2014,01,2379
2,753899,2,Los Angeles-Long Beach-Anaheim,CA,2014,01,1831
3,394463,3,Chicago,IL,2014,01,1511
4,394514,4,Dallas-Fort Worth,TX,2014,01,1199
...,...,...,...,...,...,...,...
1267,394995,119,Port St. Lucie,FL,2014,12,1279
1268,394602,159,Fort Collins,CO,2014,12,1329
1269,394405,162,Boulder,CO,2014,12,1634
1270,394645,177,Greeley,CO,2014,12,1301


In [55]:
#y14_df[['Price']].describe()

In [56]:
ry15_df = rent_df.loc[rent_df["Year"] == "2015"]
ry15_df

,RegionID,SizeRank,State,City,Year,Month,Price
1272,102001,0,United States,None,2015,01,1458
1273,394913,1,New York,NY,2015,01,2518
1274,753899,2,Los Angeles-Long Beach-Anaheim,CA,2015,01,2024
1275,394463,3,Chicago,IL,2015,01,1573
1276,394514,4,Dallas-Fort Worth,TX,2015,01,1285
...,...,...,...,...,...,...,...
2539,394995,119,Port St. Lucie,FL,2015,12,1358
2540,394602,159,Fort Collins,CO,2015,12,1417
2541,394405,162,Boulder,CO,2015,12,1764
2542,394645,177,Greeley,CO,2015,12,1393


In [57]:
ry16_df = rent_df.loc[rent_df["Year"] == "2016"]
ry16_df

,RegionID,SizeRank,State,City,Year,Month,Price
2544,102001,0,United States,None,2016,01,1525
2545,394913,1,New York,NY,2016,01,2613
2546,753899,2,Los Angeles-Long Beach-Anaheim,CA,2016,01,2168
2547,394463,3,Chicago,IL,2016,01,1629
2548,394514,4,Dallas-Fort Worth,TX,2016,01,1352
...,...,...,...,...,...,...,...
3811,394995,119,Port St. Lucie,FL,2016,12,1441
3812,394602,159,Fort Collins,CO,2016,12,1454
3813,394405,162,Boulder,CO,2016,12,1809
3814,394645,177,Greeley,CO,2016,12,1441


In [58]:
ry17_df = rent_df.loc[rent_df["Year"] == "2017"]
ry17_df

,RegionID,SizeRank,State,City,Year,Month,Price
3816,102001,0,United States,None,2017,01,1579
3817,394913,1,New York,NY,2017,01,2663
3818,753899,2,Los Angeles-Long Beach-Anaheim,CA,2017,01,2295
3819,394463,3,Chicago,IL,2017,01,1667
3820,394514,4,Dallas-Fort Worth,TX,2017,01,1418
...,...,...,...,...,...,...,...
5083,394995,119,Port St. Lucie,FL,2017,12,1523
5084,394602,159,Fort Collins,CO,2017,12,1488
5085,394405,162,Boulder,CO,2017,12,1866
5086,394645,177,Greeley,CO,2017,12,1514


In [59]:
ry18_df = rent_df.loc[rent_df["Year"] == "2018"]
ry18_df

,RegionID,SizeRank,State,City,Year,Month,Price
5088,102001,0,United States,None,2018,01,1629
5089,394913,1,New York,NY,2018,01,2700
5090,753899,2,Los Angeles-Long Beach-Anaheim,CA,2018,01,2400
5091,394463,3,Chicago,IL,2018,01,1697
5092,394514,4,Dallas-Fort Worth,TX,2018,01,1468
...,...,...,...,...,...,...,...
6355,394995,119,Port St. Lucie,FL,2018,12,1606
6356,394602,159,Fort Collins,CO,2018,12,1540
6357,394405,162,Boulder,CO,2018,12,1932
6358,394645,177,Greeley,CO,2018,12,1585


In [60]:
ry19_df = rent_df.loc[rent_df["Year"] == "2019"]
ry19_df

,RegionID,SizeRank,State,City,Year,Month,Price
6360,102001,0,United States,None,2019,01,1690
6361,394913,1,New York,NY,2019,01,2755
6362,753899,2,Los Angeles-Long Beach-Anaheim,CA,2019,01,2500
6363,394463,3,Chicago,IL,2019,01,1736
6364,394514,4,Dallas-Fort Worth,TX,2019,01,1518
...,...,...,...,...,...,...,...
7627,394995,119,Port St. Lucie,FL,2019,12,1682
7628,394602,159,Fort Collins,CO,2019,12,1579
7629,394405,162,Boulder,CO,2019,12,1984
7630,394645,177,Greeley,CO,2019,12,1630


In [61]:
ry20_df = rent_df.loc[rent_df["Year"] == "2020"]
ry20_df

,RegionID,SizeRank,State,City,Year,Month,Price
7632,102001,0,United States,None,2020,01,1742
7633,394913,1,New York,NY,2020,01,2799
7634,753899,2,Los Angeles-Long Beach-Anaheim,CA,2020,01,2568
7635,394463,3,Chicago,IL,2020,01,1774
7636,394514,4,Dallas-Fort Worth,TX,2020,01,1561
...,...,...,...,...,...,...,...
8899,394995,119,Port St. Lucie,FL,2020,12,1804
8900,394602,159,Fort Collins,CO,2020,12,1614
8901,394405,162,Boulder,CO,2020,12,2017
8902,394645,177,Greeley,CO,2020,12,1677


In [62]:
ry21_df = rent_df.loc[rent_df["Year"] == "2021"]
ry21_df

,RegionID,SizeRank,State,City,Year,Month,Price
8904,102001,0,United States,None,2021,01,1748
8905,394913,1,New York,NY,2021,01,2552
8906,753899,2,Los Angeles-Long Beach-Anaheim,CA,2021,01,2553
8907,394463,3,Chicago,IL,2021,01,1728
8908,394514,4,Dallas-Fort Worth,TX,2021,01,1587
...,...,...,...,...,...,...,...
9005,394995,119,Port St. Lucie,FL,2021,01,1815
9006,394602,159,Fort Collins,CO,2021,01,1618
9007,394405,162,Boulder,CO,2021,01,2020
9008,394645,177,Greeley,CO,2021,01,1681


In [63]:
## housing data cleaning##

In [64]:
#read data
house_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,1996-01-31,1996-02-29,1996-03-31,1996-04-30,1996-05-31,...,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31
0,102001,0,United States,Country,NaN,106811.0,106838.0,106888.0,107014.0,107156.0,...,250069.0,251258.0,252506.0,254019.0,255964.0,258235.0,260729.0,263427.0,266222.0,269039.0
1,394913,1,"New York, NY",Msa,NY,187591.0,187153.0,186875.0,186342.0,186025.0,...,483751.0,485191.0,486992.0,489417.0,492642.0,496858.0,501651.0,506833.0,511880.0,516732.0
2,753899,2,"Los Angeles-Long Beach-Anaheim, CA",Msa,CA,186111.0,186371.0,186390.0,186499.0,186473.0,...,698120.0,699783.0,701018.0,705448.0,712728.0,720482.0,727461.0,734522.0,741757.0,748532.0
3,394463,3,"Chicago, IL",Msa,IL,163099.0,162799.0,162404.0,161955.0,161353.0,...,245195.0,245715.0,246350.0,247476.0,248977.0,250996.0,253227.0,255446.0,257610.0,259459.0
4,394514,4,"Dallas-Fort Worth, TX",Msa,TX,111373.0,111437.0,111596.0,111914.0,112257.0,...,256755.0,257898.0,259226.0,260751.0,262469.0,264207.0,266483.0,268773.0,271372.0,273348.0


In [65]:
#depivot the table from the time series
house_df = pd.melt(house_df, id_vars=['RegionID', 'RegionName','SizeRank','RegionType','StateName'], var_name='Date', value_name='Price')

In [66]:
house_df

,RegionID,RegionName,SizeRank,RegionType,StateName,Date,Price
0,102001,United States,0,Country,NaN,1996-01-31,106811.0
1,394913,"New York, NY",1,Msa,NY,1996-01-31,187591.0
2,753899,"Los Angeles-Long Beach-Anaheim, CA",2,Msa,CA,1996-01-31,186111.0
3,394463,"Chicago, IL",3,Msa,IL,1996-01-31,163099.0
4,394514,"Dallas-Fort Worth, TX",4,Msa,TX,1996-01-31,111373.0
...,...,...,...,...,...,...,...
274808,394767,"Lamesa, TX",929,Msa,TX,2021-01-31,78762.0
274809,753874,"Craig, CO",930,Msa,CO,2021-01-31,195374.0
274810,394968,"Pecos, TX",931,Msa,TX,2021-01-31,129578.0
274811,395188,"Vernon, TX",932,Msa,TX,2021-01-31,68432.0


In [67]:
#check for null values
house_df.isnull().sum()

RegionID          0
RegionName        0
SizeRank          0
RegionType        0
StateName       301
Date              0
Price         51625
dtype: int64

In [68]:
#drop the null values
house_df = house_df.dropna()

In [69]:
house_df.isnull().sum()

RegionID      0
RegionName    0
SizeRank      0
RegionType    0
StateName     0
Date          0
Price         0
dtype: int64

In [70]:
#check data types
house_df.dtypes

RegionID        int64
RegionName     object
SizeRank        int64
RegionType     object
StateName      object
Date           object
Price         float64
dtype: object

In [71]:
#convert data types for further analysis
house_df['Price'] = house_df['Price'].astype(float).astype(int)

In [72]:
house_df

,RegionID,RegionName,SizeRank,RegionType,StateName,Date,Price
1,394913,"New York, NY",1,Msa,NY,1996-01-31,187591
2,753899,"Los Angeles-Long Beach-Anaheim, CA",2,Msa,CA,1996-01-31,186111
3,394463,"Chicago, IL",3,Msa,IL,1996-01-31,163099
4,394514,"Dallas-Fort Worth, TX",4,Msa,TX,1996-01-31,111373
5,394974,"Philadelphia, PA",5,Msa,PA,1996-01-31,120570
...,...,...,...,...,...,...,...
274808,394767,"Lamesa, TX",929,Msa,TX,2021-01-31,78762
274809,753874,"Craig, CO",930,Msa,CO,2021-01-31,195374
274810,394968,"Pecos, TX",931,Msa,TX,2021-01-31,129578
274811,395188,"Vernon, TX",932,Msa,TX,2021-01-31,68432


In [73]:
#split Date into Year and Month columns 
house_df[['Year','Month',"date"]] = house_df.Date.str.split("-",expand=True)
#split RegionName into State and City columns 
house_df[['State','City']] = house_df.RegionName.str.split(",",expand=True)
house_df

,RegionID,RegionName,SizeRank,RegionType,StateName,Date,Price,Year,Month,date,State,City
1,394913,"New York, NY",1,Msa,NY,1996-01-31,187591,1996,01,31,New York,NY
2,753899,"Los Angeles-Long Beach-Anaheim, CA",2,Msa,CA,1996-01-31,186111,1996,01,31,Los Angeles-Long Beach-Anaheim,CA
3,394463,"Chicago, IL",3,Msa,IL,1996-01-31,163099,1996,01,31,Chicago,IL
4,394514,"Dallas-Fort Worth, TX",4,Msa,TX,1996-01-31,111373,1996,01,31,Dallas-Fort Worth,TX
5,394974,"Philadelphia, PA",5,Msa,PA,1996-01-31,120570,1996,01,31,Philadelphia,PA
...,...,...,...,...,...,...,...,...,...,...,...,...
274808,394767,"Lamesa, TX",929,Msa,TX,2021-01-31,78762,2021,01,31,Lamesa,TX
274809,753874,"Craig, CO",930,Msa,CO,2021-01-31,195374,2021,01,31,Craig,CO
274810,394968,"Pecos, TX",931,Msa,TX,2021-01-31,129578,2021,01,31,Pecos,TX
274811,395188,"Vernon, TX",932,Msa,TX,2021-01-31,68432,2021,01,31,Vernon,TX


In [74]:
#adjust the data frame columns
house_df = house_df[['RegionID','SizeRank','State','City','Year', 'Month','Price']]
house_df

,RegionID,SizeRank,State,City,Year,Month,Price
1,394913,1,New York,NY,1996,01,187591
2,753899,2,Los Angeles-Long Beach-Anaheim,CA,1996,01,186111
3,394463,3,Chicago,IL,1996,01,163099
4,394514,4,Dallas-Fort Worth,TX,1996,01,111373
5,394974,5,Philadelphia,PA,1996,01,120570
...,...,...,...,...,...,...,...
274808,394767,929,Lamesa,TX,2021,01,78762
274809,753874,930,Craig,CO,2021,01,195374
274810,394968,931,Pecos,TX,2021,01,129578
274811,395188,932,Vernon,TX,2021,01,68432


In [75]:
# Filtered df to match the rent data
house_df['Year'] = house_df['Year'].astype(int)
house_filtered = house_df[house_df['Year'] >= 2014]  
house_filtered.head()

,RegionID,SizeRank,State,City,Year,Month,Price
197209,394913,1,New York,NY,2014,01,393733
197210,753899,2,Los Angeles-Long Beach-Anaheim,CA,2014,01,515143
197211,394463,3,Chicago,IL,2014,01,197262
197212,394514,4,Dallas-Fort Worth,TX,2014,01,161947
197213,394974,5,Philadelphia,PA,2014,01,212225


In [76]:
house_df

,RegionID,SizeRank,State,City,Year,Month,Price
1,394913,1,New York,NY,1996,01,187591
2,753899,2,Los Angeles-Long Beach-Anaheim,CA,1996,01,186111
3,394463,3,Chicago,IL,1996,01,163099
4,394514,4,Dallas-Fort Worth,TX,1996,01,111373
5,394974,5,Philadelphia,PA,1996,01,120570
...,...,...,...,...,...,...,...
274808,394767,929,Lamesa,TX,2021,01,78762
274809,753874,930,Craig,CO,2021,01,195374
274810,394968,931,Pecos,TX,2021,01,129578
274811,395188,932,Vernon,TX,2021,01,68432


In [78]:
# Saved to csv files
rent_df.to_csv('rent_cleaned.csv', index=False, header=True)
house_df.to_csv('house_cleaned.csv', index=False, header=True)
house_filtered.to_csv('house_filtered.csv', index=False, header=True)